# Continuous Control

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [ ]:
from unityagents import UnityEnvironment

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Reacher.app"`
- **Windows** (x86): `"path/to/Reacher_Windows_x86/Reacher.exe"`
- **Windows** (x86_64): `"path/to/Reacher_Windows_x86_64/Reacher.exe"`
- **Linux** (x86): `"path/to/Reacher_Linux/Reacher.x86"`
- **Linux** (x86_64): `"path/to/Reacher_Linux/Reacher.x86_64"`
- **Linux** (x86, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86"`
- **Linux** (x86_64, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86_64"`

For instance, if you are using a Mac, then you downloaded `Reacher.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Reacher.app")
```

In [ ]:
env = UnityEnvironment(file_name='Reacher.app')

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [ ]:
env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
while True:
    actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

When finished, you can close the environment.

In [ ]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [ ]:
from unityagents import UnityEnvironment
env = UnityEnvironment(file_name='Reacher.app')

In [ ]:
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

In [ ]:
import torch    
import torch.autograd as autograd         
import torch.nn as nn                     
import torch.nn.functional as F           
import torch.optim as optim               
import torch.distributions

import pandas as pd
import numpy as np

import holoviews as hv
from holoviews import opts
from holoviews.streams import Pipe, Buffer

import streamz
import streamz.dataframe

import pdb, gym

hv.extension('bokeh')

In [ ]:
# Setup training progress output
def init_training_progress():

    max_length = 1000000
    rolling_size = 25

    training_stream = streamz.Stream()
    example = pd.DataFrame({'x': [0]}, index=[0])

    training_sdf = streamz.dataframe.DataFrame(training_stream, example=example)

    training_raw_buffer = Buffer(training_sdf, length=max_length)
    training_smooth_buffer = Buffer(training_sdf.x.rolling(rolling_size).median())

    training_raw_dmap = hv.DynamicMap(hv.Curve, streams=[training_raw_buffer]).relabel('raw')
    training_smooth_dmap = hv.DynamicMap(hv.Curve, streams=[training_smooth_buffer]).relabel('smooth')
    
    return training_stream, training_raw_dmap, training_smooth_dmap

In [ ]:
class PolicyNet(nn.Module):
    def __init__(self):
        super(PolicyNet, self).__init__()

        # 33 continous input state variables
        state_size = 33
        
        # 1 continuous output action, mean and sigma (std. dev.) for each
        actions_size = 1
        
        # Hidden layers
        hidden_size = 64
        self.fc1 = nn.Linear(state_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        
        # Output layer of action means
        self.fc4 = nn.Linear(hidden_size, actions_size)
        
        # Standard deviations approximated seperately
        self.register_parameter('log_sigma', None)
        self.log_sigma = nn.Parameter(torch.ones(actions_size), requires_grad=True)        

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        
        means = torch.tanh(self.fc4(x))
        sigmas = torch.exp(self.log_sigma).expand(means.shape)
        
        return means, sigmas

In [ ]:
class Agent():
    
    # This agent handles multidimensional continuous action spaces by maintaining
    # multiple sub-agents, one for each action dimension.
    
    def __init__(self, action_size=4, lr=1e-2):
        
        # Create sub agents for each action dimension
        self.subagents = [SubAgent(lr=lr) for i in range(action_size)]
        
    def reset_before_episode(self):
        [subagent.reset_before_episode() for subagent in self.subagents]
        
    def average_episode_return(self):
        # all subagents track the same rewards/returns so we just pick one
        per_trajectory_rewards = self.subagents[0].trajectories['rewards']
        return torch.tensor(per_trajectory_rewards).sum(dim=0).mean().item()
    
    def act(self, states):
        
        # Calculate all next actions for all envs
        per_subagent_actions = []
        for subagent in self.subagents:
            
            # Get actions this agent
            subagent_actions = subagent.act(states)
            subagent_actions = np.clip(subagent_actions.numpy(), -1, 1)  
            per_subagent_actions.append(subagent_actions)
            
        # Combine actions from the n sub-agents into the required nD action vector
        actions = torch.tensor(per_subagent_actions).float()
        actions = actions.permute(1,0,2).squeeze()
        return actions
    
    def record_experience(self, states, actions, rewards, next_states):
        [subagent.record_experience(states, actions, rewards, next_states) 
         for subagent in self.subagents]
        
    def learn(self):
        [subagent.learn() for subagent in self.subagents]

In [ ]:
class SubAgent():
    
    def __init__(self, lr=1e-2):
        self.policy = PolicyNet()
        self.optimiser = optim.Adam(self.policy.parameters(), lr=lr)

    def reset_before_episode(self):
        # We train on multiple environments in parallel, therefore each
        # of these lists represents multiple trajectories
        self.trajectories = {
            "states": [],
            "actions": [],
            "rewards": [],
            "log_probs": []
        }

    def act(self, states):
        m = self.action_distribution_for_states(states)
        actions = m.sample()
        self.trajectories['states'].append(states)
        self.trajectories['actions'].append(actions)
        self.trajectories['log_probs'].append(m.log_prob(actions))
        return actions
    
    def action_distribution_for_states(self, states):
        means, sigmas = self.policy.forward(states)
        return torch.distributions.Normal(means, sigmas)
    
    def record_experience(self, states, actions, rewards, next_states):
        self.trajectories['rewards'].append(rewards)
    
    def learn(self):
        
        # Calculate future return at each step
        # This is just the cummulative sum of rewards calculated backwards from the episode end
        per_trajectory_future_return = torch.flip(
            torch.cumsum(
                torch.flip(
                    torch.tensor(self.trajectories['rewards']),
                    dims=(0,)),
                dim=0
            ),
            dims=(0,)
        )
        
        # Normalise future return at each step
        per_trajectory_future_return -= per_trajectory_future_return.mean(dim=1).unsqueeze(1)
        per_trajectory_future_return /= per_trajectory_future_return.std(dim=1).unsqueeze(1) + 1.0e-10

        # Prepare probabilities of selecting each action, at each timestep, for each trajectory
        # These were recording as the agent selected its actions
        # We don't differentiate with respect to these, hence we detach them
        old_log_probs = torch.stack(self.trajectories['log_probs']).squeeze(2).detach()
        old_probs = torch.exp(old_log_probs)
        
        # Run through PPO epochs
        for ppo_epoch in range(10):
        
            # Get new probabilities of selecting each action, at each timestep, for each trajectory
            # Here we use the current policy, not the one used during the actions themselves
            states_actions_pairs = zip(self.trajectories['states'], self.trajectories['actions'])
            log_probs_list = [self.action_distribution_for_states(states).log_prob(actions)
                                        for (states, actions) in states_actions_pairs]
            log_probs = torch.stack(log_probs_list).squeeze(2)
            probs = torch.exp(log_probs)

            # Define loss using PPO surrogate
            per_trajectory_is_ratio = (probs/old_probs)
            per_trajectory_surrogate = (per_trajectory_is_ratio * per_trajectory_future_return).sum(dim=0)
            loss = -torch.mean(per_trajectory_surrogate)

             # Update model
            self.optimiser.zero_grad()
            loss.backward()
            self.optimiser.step()
    

In [ ]:
def play_episode_and_learn(envs, agent, time_max=999):
    
    # Run concurrent episode on all environments
    env_info = env.reset(train_mode=True)[brain_name]
    agent.reset_before_episode()
    states = torch.from_numpy(env_info.vector_observations).float()
    for time in range(time_max):
        
        # Calculate actions for all envs
        actions = agent.act(states)
        
        # Run through the envs in parallel
        env_info = env.step(actions.numpy())[brain_name]
        next_states = torch.from_numpy(env_info.vector_observations).float()
        rewards = env_info.rewards
        dones = env_info.local_done
        
        # Record the experience tuple with the agent
        agent.record_experience(states, actions, rewards, next_states)
        
        # Advance
        states = next_states

        # We want rectangular input to network, so if any finish early we finish all early
        if any(dones):
            break
    
    # Instruct agent to learn from the episode
    agent.learn()

In [ ]:
# Create policy and agent
agent = Agent(action_size=brain.vector_action_space_size,
              lr=3e-4)

# Create returns record, this will stay consistent across runs
time_index = 0
average_returns = []

In [ ]:
# Display training progress
stream, smooth, raw = init_training_progress()
layout = (smooth * raw)
layout.opts(
    opts.Curve(width=900, height=300, show_grid=True, tools=['hover'])
)

In [ ]:
# Train
episode_max = 10
time_max = 1024 # 2048
for i_episode in range(episode_max):
    
    play_episode_and_learn(env, agent, time_max=time_max)

    stream.emit( pd.DataFrame({'x': agent.average_episode_return()}, index=[time_index]) )
    time_index += 1

In [ ]:
# Train
episode_max = 100
time_max = 1024 # 2048
for i_episode in range(episode_max):
    
    play_episode_and_learn(env, agent, time_max=time_max)

    stream.emit( pd.DataFrame({'x': agent.average_episode_return()}, index=[time_index]) )
    time_index += 1